# Processing NeXus Choppers

When choppers are loaded from NeXus, they typically contain a number of fields that need to be processed before they can be used for computing wavelength ranges, etc.
This guide shows how to extract the relevant data from such a NeXus chopper and create a [scippneutron.chopper.DiskChopper](../../generated/modules/scippneutron.chopper.disk_chopper.DiskChopper.rst) object.

In [ ]:
import scipp as sc

Here, we use fake data which roughly represents what a real chopper loaded from NeXus looks like.
ScippNeutron has a function for generating this data:

In [ ]:
from scippneutron.data import chopper_mockup

chopper = chopper_mockup()
chopper

We can see that there is some information about the slits and geometry of the chopper as well as some timing-related data.
Take a look at the [NXdisk_chopper](https://manual.nexusformat.org/classes/base_classes/NXdisk_chopper.html) documentation for an overview of the fields.

In this case, there already is a `position`.
This typically needs to be computed first, see [scippnexus.compute_positions](https://scipp.github.io/scippnexus/generated/functions/scippnexus.compute_positions.html).

Some fields are nested data groups which happens when a NeXus file contains `NXlog`s.
We can extract the relevant arrays from them using [extract_chopper_from_nexus](../../generated/modules/scippneutron.chopper.nexus_chopper.extract_chopper_from_nexus.rst):

In [ ]:
from scippneutron.chopper import extract_chopper_from_nexus

chopper = extract_chopper_from_nexus(chopper)
chopper

Some data varies with time, which can complicate the data processing.
Instead, we compute corresponding time-independent quantities from the raw chopper data.

## Identify In-phase Regions

Frame unwrapping is only feasible when the chopper is in-phase with the neutron source pulses because, otherwise, the wavelength frames vary pulse-by-pulse.
To identify regions where the chopper is in-phase, we first find plateaus in the `rotation_speed` which is the rotation frequency of the chopper.

In [ ]:
rotation_speed = chopper['rotation_speed']
rotation_speed.name = 'rotation_speed'
rotation_speed

The chopper has a long region of near-constant rotation speed surrounded by spin-up and spin-down regions:

In [ ]:
rotation_speed.plot(markersize=2)

We use [find_plateaus](../../generated/modules/scippneutron.chopper.filtering.find_plateaus.rst) and [collapse_plateaus](../../generated/modules/scippneutron.chopper.filtering.collapse_plateaus.rst) to find those plateaus.
Note the `atol` and `min_n_points` parameters, they need to be tuned for the specific input data.

<div class="alert alert-warning">

**Warning**

``find_plateaus`` can potentially falsely identify regions with a small but steady slope as a plateau.
See the function's documentation for details.

</div>

In [ ]:
from scippneutron.chopper import collapse_plateaus, find_plateaus

plateaus = find_plateaus(rotation_speed, atol=sc.scalar(1e-3, unit='Hz / s'), min_n_points=10)
plateaus = collapse_plateaus(plateaus)
plateaus

`find_plateaus` found two plateaus that we can plot with the following helper function:

In [ ]:
def plot_plateaus(raw_data: sc.DataArray, plateaus: sc.DataArray) -> None:
    to_plot = sc.DataGroup({'Rotation Speed': raw_data})
    for plateau in plateaus:
        i = plateau.coords['plateau'].value
        to_plot[f'Plateau {i}'] = sc.DataArray(
            plateau.data.broadcast(dims=['time'], shape=[2]),
            coords={'time': plateau.coords['time']}
        )
    return to_plot.plot(
        ls={f'Plateau {i}': '-' for i in range(len(plateaus))},
        marker={f'Plateau {i}': '|' for i in range(len(plateaus))},
        markersize={'Rotation Speed': 2},
    )

In [ ]:
plot_plateaus(rotation_speed, plateaus)

In this case, the source has a frequency of 14Hz which means that plateau 0 is in phase.
But plateau 1 is not, it is a short region where the chopper slowed down before fully stopping.

We can use [filter_in_phase](../../generated/modules/scippneutron.chopper.filtering.filter_in_phase.rst) to remove all out-of-phase plateaus:

In [ ]:
pulse_frequency = sc.scalar(14.0, unit='Hz')

In [ ]:
from scippneutron.chopper import filter_in_phase

frequency_in_phase = filter_in_phase(
    plateaus,
    reference=pulse_frequency,
    rtol=sc.scalar(1e-3))
frequency_in_phase

In [ ]:
plot_plateaus(rotation_speed, frequency_in_phase)

## Extract Plateau

Since there is only one plateau left, we can simply index into it to get the chopper frequency:

In [ ]:
frequency = frequency_in_phase['plateau', 0]
frequency

Next, we need the TDC timestamps for the in-phase region:

In [ ]:
tdc = chopper['top_dead_center']
tdc

In [ ]:
low = frequency.coords['time'][0]
high = frequency.coords['time'][1]
tdc_in_phase = tdc[(tdc > low) & (tdc < high)]
tdc_in_phase

We can check that the rate at which the TDC triggers is indeed close to 14Hz.

In [ ]:
diff = tdc_in_phase[1:] - tdc_in_phase[:-1]
rate = 1 / diff.to(unit='s', dtype='float64')
rate.min(), rate.max()

## Compute Chopper Phase

`DiskChopper` does not use TDC directly for time calculations but instead the chopper phase $\phi$.
According to the [disk chopper docs](../../generated/modules/scippneutron.chopper.disk_chopper.rst), the phase is defined as
$$\phi = \omega (t_0 + \delta_t - T_0),$$
where $t_0$ is a TDC timestamp and $T_0$ a pulse time.

We already determined the TDC timestamps above.
In practice, we would get $T_0$ from the input NeXus file, but here, we simply make one up:

In [ ]:
pulse_time = sc.datetime('2023-01-19T08:12:03.442912915', unit='ns')

<div class="alert alert-info">

**Note**

The pulse time is typically an array of timestamps and it can be difficult to determine which pulse goes with which chopper period.
While the choice is technically arbitrary, the times calculated by `DiskChopper` are relative to the chosen pulse time.

If the chopper rotates at the pulse frequency or an integer multiple of it, we can select any pulse time and TDC timestamp and simply use `phase = phase % (2 * sc.constants.pi)` below.
This corresponds to selecting the pulse and TDC times that are closest to each other.

</div>

(We multiply by 1 rad to get the proper `rad*Hz` unit in `omega`.)

In [ ]:
omega = 2 * sc.constants.pi * frequency.data * sc.scalar(1, unit='rad')
phase = omega * (tdc_in_phase[0] + chopper['delay'].data - pulse_time)
phase = phase.to(unit='rad')
phase

## Build `DiskChopper`

Finally, we can assemble all data into a [scippneutron.chopper.DiskChopper](../../generated/modules/scippneutron.chopper.disk_chopper.DiskChopper.rst) object.

The rotation speed gets rounded (resulting in 14Hz) because `DiskChopper` requires it to be a near exact integer multiple of the pulse frequency or vice versa:

- `rotation_speed = N * pulse_frequency`
- `rotation_speed = pulse_frequency / N`

where `N` is an integer number.

In [ ]:
processed = chopper.copy()
processed['rotation_speed'] = sc.round(frequency.data)
processed['phase'] = phase

The input data does not contain a beam position (the angle between the beam and TDC).
This probably means that it is 0.
But since `DiskChopper` does not make that assumption we have to be explicit:

In [ ]:
processed['beam_position'] = sc.scalar(0.0, unit='rad')

In [ ]:
from scippneutron.chopper import DiskChopper

disk_chopper = DiskChopper.from_nexus(processed)
disk_chopper